### Load dataset and save IRs

In [ ]:
from diff_gfdn.dataloader import ThreeRoomDataset, load_dataset
from diff_gfdn.config.config import DiffGFDNConfig
from diff_gfdn.model import DiffGFDN
from diff_gfdn.utils import get_response, db, is_unitary, absorption_to_gain_per_sample
from diff_gfdn.losses import get_stft_torch, get_edr_from_stft
from run_model import load_and_validate_config

from pathlib import Path
from typing import Tuple, Optional
from numpy.typing import ArrayLike
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torchaudio
import soundfile as sf
from loguru import logger

### Helper functions

In [ ]:
def plot_spectrogram(S: torch.tensor, freqs: ArrayLike, time_frames: ArrayLike, title:Optional[str]=None):
    plt.figure()
    plt.imshow(db(np.abs(S)).cpu().detach().numpy(), aspect='auto', origin='lower',
    extent=[time_frames.min(), time_frames.max(), freqs.min(), freqs.max()])
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    cbar = plt.colorbar()
    cbar.set_label('dB')
    if title is not None:
        plt.title(title)
    plt.show()


def plot_edr(h: torch.tensor, 
             fs: float, 
             win_size:int=2**9, 
             hop_size:int=2**8, 
             title:Optional[str]=None) -> Tuple[torch.tensor, ArrayLike, ArrayLike]:
    S, freqs, time_frames = get_stft_torch(h, fs, win_size=win_size, hop_size=hop_size, nfft=win_size, freq_axis=0)
    edr = get_edr_from_stft(S)
    plot_spectrogram(edr, freqs, time_frames,title)
    return edr

### Read config files and dataset

In [ ]:
config_path = '../data/config/'
config_file = config_path + 'antialiasing_sample_outside_unit_circle.yml'
config_dict = load_and_validate_config(config_file,
                                       DiffGFDNConfig)
room_data = ThreeRoomDataset(Path(config_dict.room_dataset_path).resolve(), save_irs=False)

# add number of groups to the config dictionary
config_dict = config_dict.copy(update={"num_groups": room_data.num_rooms})

if config_dict.sample_rate != room_data.sample_rate:
    logger.warn("Config sample rate does not match data, alterning it")
    config_dict.sample_rate = sample_rate

# get the training config
trainer_config = config_dict.trainer_config

# prepare the training and validation data for DiffGFDN
train_dataset, valid_dataset = load_dataset(
    room_data, trainer_config.device, trainer_config.train_valid_split,
    trainer_config.batch_size)

### Check output data and compare with true IR

In [ ]:
# initialise the model
model = DiffGFDN(room_data.sample_rate, room_data.num_rooms,
                 config_dict.delay_length_samps,
                 room_data.room_dims,
                 trainer_config.device, 
                 config_dict.feedback_loop_config,
                 config_dict.output_filter_config,
                 config_dict.use_absorption_filters,
                 common_decay_times=room_data.common_decay_times,
                 band_centre_hz=room_data.band_centre_hz
                )

In [ ]:
audio_directory  = Path("../audio/")
checkpoint_dir = Path(trainer_config.train_dir + 'checkpoints/').resolve()
max_epochs = trainer_config.max_epochs
save_ir_dir = Path(trainer_config.ir_dir).resolve() 
save_ir = False
plot_ir = not save_ir  

In [ ]:
for epoch in range(max_epochs):
    # load the trained weights for the particular epoch
    checkpoint = torch.load(f'{checkpoint_dir}/model_e{epoch}.pt', weights_only=True)
    # Load the trained model state
    model.load_state_dict(checkpoint)
    # in eval mode, no gradients are calculated
    model.eval()
    
    for data in train_dataset:
        position = data['listener_position']
        H, h = get_response(data, model)
        
        for num_pos in range(position.shape[0]):
            filename = f'ir_({position[num_pos,0]:.2f}, {position[num_pos, 1]:.2f}, {position[num_pos, 2]:.2f}).wav'
            
            # find the true IR corresponding to this position
            filepath_true = os.path.join(Path(audio_directory/'true').resolve(), filename)
            h_true = torch.from_numpy(sf.read(filepath_true)[0])

            if plot_ir:
                # plot the EDRs of the true and estimated
                plot_edr(h_true, model.sample_rate, title=f'True RIR EDR, epoch={epoch}')
                plot_edr(h[num_pos, :], model.sample_rate, title=f'Estimated RIR EDR, epoch={epoch}')
        
                plt.figure()
                plt.plot(torch.stack((h_true, h[num_pos, :len(h_true)]), dim=-1))
                plt.show()

            if save_ir and (epoch == max_epochs - 1):
                outer_loop_break = False
                filepath = os.path.join(save_ir_dir, filename)
                torchaudio.save(filepath,
                            torch.stack((h[num_pos, :], h[num_pos, :]),
                                        dim=1).cpu(),
                            int(model.sample_rate),
                            bits_per_sample=32,
                            channels_first=False)
            else:
                outer_loop_break = True
                break
                
        if outer_loop_break:
            break

### Get the final trained parameters and investigate them

In [ ]:
from scipy.io import loadmat
param_path = Path('../output/mps/parameters_opt.mat')
opt_params = loadmat(param_path.resolve())
print(opt_params.keys())

#### Observe the individual mixing matrices

In [ ]:
from importlib import reload
import diff_gfdn
reload(diff_gfdn.utils)
from diff_gfdn.utils import is_unitary

M_list = opt_params['feedback_loop.M']
num_groups = model.num_groups
fig, ax = plt.subplots(num_groups, 1, figsize=(6,6))

for i in range(num_groups):
    M = torch.from_numpy(M_list[i, ...])
    with torch.no_grad():
        M_ortho = model.feedback_loop.ortho_param(M)
    ax[i].matshow(torch.abs(M_ortho))
    ax[i].set_title(f'Room {i}')
    is_ortho, max_val = is_unitary(M_ortho)
    print(max_val)
plt.tight_layout()

#### Observe the input gains, coupling matrix and the coupled mixing matrix

In [ ]:
alpha = opt_params['feedback_loop.alpha'][0]
coupled_feedback_matrix = opt_params['coupled_feedback_matrix']
assert is_unitary(torch.from_numpy(coupled_feedback_matrix))[0]

input_gains = opt_params['input_gains'][0]
print(np.linalg.norm(input_gains))

with torch.no_grad():
    coupling_matrix =model.feedback_loop.nd_unitary(torch.from_numpy(alpha), room_data.num_rooms)
    unit_flag, max_val = is_unitary(coupling_matrix)
    assert unit_flag
    
plt.figure()
plt.subplot(211)
plt.matshow(torch.abs(coupling_matrix), fignum=False)
plt.colorbar()
plt.title('Coupling matrix')
plt.subplot(212)
plt.matshow(np.abs(coupled_feedback_matrix), fignum=False)
plt.colorbar()
plt.title('Coupled feedback matrix')
plt.tight_layout()

#### Plot the output filter response for a few positions

In [ ]:
from scipy.signal import sosfreqz
output_biquad_coeffs = opt_params['output_biquad_coeffs']
delays = opt_params['delays'][0]

for b in range(len(output_biquad_coeffs)):
    fig, ax = plt.subplots()  # Create a new figure and axis for each outer loop iteration
    for n in range(len(delays)):
        cur_biquad_coeffs = output_biquad_coeffs[b][n]
        # ensure a0 = 1 (needed by scipy)
        for k in range(cur_biquad_coeffs.shape[0]):
            cur_biquad_coeffs[k,:] /= cur_biquad_coeffs[k, 3]

        freqs, filt_response = sosfreqz(cur_biquad_coeffs, worN=2**9, fs=room_data.sample_rate)
        ax.semilogx(freqs, 20*np.log10(np.abs(filt_response)), label=f'Line {n}')
        
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Magnitude (dB)')
    ax.set_title(f'Output filter for position {b}')
    ax.legend()